In [1]:
import os 
import sys
import pygwalker as pyg
import pandas as pd

In [2]:

#Importing Configs
# Define the path where config.py is located
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local
Sentiment_models = config.Sentiment_models

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
#Model_Folder = config.texts
Model_Folder = config.Sentiment_models
Model_Folder = Model_Folder + Model_Subfolder
Model_Folder = config.Sentiment_models

Body_2 = config.Body_2
Model_2 = config.Model_2
Model_Subfolder_2 = f'/{Body_2} Texts/{Model_2}'
Model_Folder_2 = config.texts
Model_Folder_2 = Model_Folder_2 + Model_Subfolder_2

/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling
/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling
/Users/kylenabors/Documents
/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [3]:
df = pd.read_csv(f"{Sentiment_models}/{Body}_{Model}_advanced_sentiment_texts.csv")
df_2 = pd.read_csv(f"{Sentiment_models}/{Body_2}_{Model_2}_advanced_sentiment_texts.csv")

In [4]:
dummies = pd.get_dummies(df['sentiment'], dtype=float)
df = df.join(dummies)
dummies_2 = pd.get_dummies(df_2['sentiment'], dtype=float)
df_2 = df_2.join(dummies)

In [5]:
print(df.head())
print(df_2.head())

   Unnamed: 0        date                                title  \
0           0  2023-08-31  Disinflation and the Phillips curve   
1           1  2023-08-31  Disinflation and the Phillips curve   
2           2  2023-08-31  Disinflation and the Phillips curve   
3           3  2023-08-31  Disinflation and the Phillips curve   
4           4  2023-08-31  Disinflation and the Phillips curve   

              type                                            segment  \
0  Isabel Schnabel  speech disinflation and the phillips curve spe...   
1  Isabel Schnabel  activity has moderated visibly , and forwardlo...   
2  Isabel Schnabel  but important pockets of resilience remain , e...   
3  Isabel Schnabel  headline inflation has come down , mainly on t...   
4  Isabel Schnabel  but underlying price pressures remain stubborn...   

  sentiment  confidence  neg  pos  
0       neg         0.6  1.0  0.0  
1       pos         1.0  0.0  1.0  
2       pos         1.0  0.0  1.0  
3       pos         

In [6]:
df_group = df.copy(deep=True)
df_group = df_group.drop(columns=['Unnamed: 0', 'title', 'type', 'segment', 'sentiment', 'confidence'])
df_group['monthly'] = df_group['date'].str[0:7]
df_group = df_group[['monthly', 'pos', 'neg']]
df_group = df_group.groupby('monthly', as_index = False).mean()
df_group['net'] = df_group['pos'] - df_group['neg']
df_group['net'] = df_group['net'] - df_group['net'].mean()
print(df_2.head()) 

df_2_group = df_2.copy(deep=True)
df_2_group = df_2_group.drop(columns=['Unnamed: 0', 'title', 'type', 'segment', 'sentiment', 'confidence'])
df_2_group['date'] = df_2_group['date'].astype(str)
df_2_group['monthly'] = df_2_group['date'].str[0:7]
df_2_group = df_2_group[['monthly', 'pos', 'neg']]
df_2_group = df_2_group.groupby('monthly', as_index = False).mean()
df_2_group['net'] = df_2_group['pos'] - df_group['neg']
df_2_group['net'] = df_2_group['net'] - df_2_group['net'].mean()
print(df_group.head())
print(df_2_group.head())

df_merged = df_group.merge(df_2_group, on='monthly', how='inner')
df_merged.fillna(0, inplace=True)

   Unnamed: 0        date                                      title  \
0           0  1998-11-12  Modernizing financial services regulation   
1           1  1998-11-12  Modernizing financial services regulation   
2           2  1998-11-12  Modernizing financial services regulation   
3           3  1998-11-12  Modernizing financial services regulation   
4           4  1998-11-12  Modernizing financial services regulation   

                         type  \
0  Governor Laurence H. Meyer   
1  Governor Laurence H. Meyer   
2  Governor Laurence H. Meyer   
3  Governor Laurence H. Meyer   
4  Governor Laurence H. Meyer   

                                             segment sentiment  confidence  \
0  modernizing financial services regulation it i...       pos         1.0   
1  technology has hastened the evolution some wou...       neg         1.0   
2  the distinction between financial products is ...       pos         1.0   
3  in the united states , these market developmen...    

In [7]:
print(df_merged.head())

df_merged.rename(columns={'pos_x': 'pos_1', 'neg_x': 'neg_1', 'net_x': 'net_1', 'pos_y': 'pos_2', 'neg_y': 'neg_2', 'net_y': 'net_2'}, inplace=True)

print(df_merged.head())

   monthly     pos_x     neg_x     net_x     pos_y     neg_y     net_y
0  1998-06  0.898305  0.101695  0.100682  0.835991  0.164009  0.060813
1  1998-07  0.875536  0.124464  0.055145  0.764179  0.235821 -0.033767
2  1998-09  0.880829  0.119171  0.065730  0.827381  0.172619  0.034727
3  1998-10  0.873181  0.126819  0.050433  0.831579  0.168421  0.031277
4  1998-11  0.882764  0.117236  0.069600  0.855330  0.144670  0.064611
   monthly     pos_1     neg_1     net_1     pos_2     neg_2     net_2
0  1998-06  0.898305  0.101695  0.100682  0.835991  0.164009  0.060813
1  1998-07  0.875536  0.124464  0.055145  0.764179  0.235821 -0.033767
2  1998-09  0.880829  0.119171  0.065730  0.827381  0.172619  0.034727
3  1998-10  0.873181  0.126819  0.050433  0.831579  0.168421  0.031277
4  1998-11  0.882764  0.117236  0.069600  0.855330  0.144670  0.064611


In [8]:
df_merged['net_diff'] = df_merged['net_1'] - df_merged['net_2']

In [9]:
funds = pd.read_excel('/Users/kylenabors/Documents/Database/Other Data/FedFundsRate.xlsx', sheet_name='Monthly')
print(funds.head())
energy = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/Energy Prices/US Energy CPI.csv')
print(energy.head())

        DATE  FEDFUNDS Date Adjusted
0 1999-12-31      5.45    2000-01-01
1 2000-01-31      5.73    2000-02-01
2 2000-02-29      5.85    2000-03-01
3 2000-03-31      6.02    2000-04-01
4 2000-04-30      6.27    2000-05-01
         DATE  CPIENGSL
0  2000-01-01     115.0
1  2000-02-01     118.8
2  2000-03-01     124.3
3  2000-04-01     120.9
4  2000-05-01     120.0


In [10]:
funds['Diff'] = funds['FEDFUNDS'].diff()


In [11]:
df_merged.to_csv(f"{Sentiment_models}/{Body}_{Model}_vs_{Body_2}_{Model_2}_advanced_sentiment_texts.csv", index=False)

In [12]:
gwalker = pyg.walk(df_merged)

Box(children=(HTML(value='<div id="ifr-pyg-0" style="height: auto">\n    <head>\n        <meta http-equiv="Con…

In [13]:
print(df_merged.describe())

            pos_1       neg_1       net_1       pos_2       neg_2       net_2  \
count  284.000000  284.000000  284.000000  284.000000  284.000000  284.000000   
mean     0.848709    0.151291    0.001490    0.824941    0.175059    0.000100   
std      0.038886    0.038886    0.077773    0.044460    0.044460    0.061164   
min      0.666667    0.021277   -0.362595    0.500000    0.086957   -0.345080   
25%      0.827111    0.125974   -0.041707    0.806817    0.147436   -0.032462   
50%      0.849662    0.150338    0.003396    0.831579    0.168421    0.008737   
75%      0.874026    0.172889    0.052124    0.852564    0.193183    0.042945   
max      0.978723    0.333333    0.261518    0.913043    0.500000    0.120931   

         net_diff  
count  284.000000  
mean     0.001389  
std      0.080171  
min     -0.430771  
25%     -0.036781  
50%      0.000099  
75%      0.051941  
max      0.345726  
